In [ ]:
# Residual plots of distance^2 - area for all ROIs

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

rois = ["V1", "V2", "V3", "hV4", "VO1", "VO2",
        "PHC1", "PHC2", "TO2", "TO1", "LO2", "LO1", "V3B", "V3A",
        "IPS0", "IPS1", "IPS2", "IPS3", "IPS4", "IPS5", "SPL1", "FEF"]

# Load object arrays
roi_results_distance = np.load('roi_results_dist.npy', allow_pickle=True).item()
roi_results_area     = np.load('roi_results_area.npy', allow_pickle=True).item()

z = 1.96  # for ~95% CI

residuals_by_roi = {}
se_by_roi = {}

for roi in roi_results_area.keys():
    if roi not in roi_results_distance:
        continue

    x_common = roi_results_area[roi]["x_common"]

    y_areal = roi_results_area[roi]["y_fixed"]
    y_distance = roi_results_distance[roi]["y_fixed"]

    # Residual
    residual = y_distance**2 - y_areal
    residuals_by_roi[roi] = residual

    # Standard error from CI for both y_areal and y_distance
    y_lower_areal = roi_results_area[roi]["y_lower"]
    y_upper_areal = roi_results_area[roi]["y_upper"]
    se_areal = (y_upper_areal - y_lower_areal) / (2 * z)

    y_lower_distance = roi_results_distance[roi]["y_lower"]
    y_upper_distance = roi_results_distance[roi]["y_upper"]
    se_distance = (y_upper_distance - y_lower_distance) / (2 * z)

    # Propagate SE through y_distance**2
    se_residual = np.sqrt((2 * y_distance * se_distance) ** 2 + se_areal ** 2)
    se_by_roi[roi] = se_residual

# Assign rainbow colors
n_rois = len(residuals_by_roi)
colors = cm.rainbow(np.linspace(0, 1, n_rois))

# Plot residuals with SE shading
plt.figure(figsize=(10,9))
for i, (roi, resid) in enumerate(residuals_by_roi.items()):
    se = se_by_roi.get(roi, None)
    if se is not None:
        plt.fill_between(x_common, resid - se, resid + se,
                         color=colors[i], alpha=0.2, lw=0)  # shaded band
    plt.plot(x_common, resid, label=roi, lw=2, color=colors[i])  # residual line

plt.axhline(0, color='k', linestyle='--', lw=2)
plt.xlim(0.1, 8)
plt.ylim(-100, 500)
plt.xlabel("Eccentricity (deg)", fontsize=30)
plt.ylabel("Residuals (mm²/deg²) ", fontsize=30)

# Log scale with exact ticks
ax = plt.gca()
ax.set_xscale('log')
ax.set_xticks([0.1, 1, 8])
ax.set_xticklabels(['0.1', '1', '8'], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#plt.legend(fontsize=18, ncol=2, loc='upper right')
plt.tight_layout()
plt.savefig("squared_residuals_with_se.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# # Residual plots of distance - sqrt(area) for all ROIs

z = 1.96  # for ~95% CI

residuals_by_roi = {}
se_by_roi = {}

for roi in roi_results_area.keys():
    if roi not in roi_results_distance:
        continue

    x_common = roi_results_area[roi]["x_common"]

    y_areal = roi_results_area[roi]["y_fixed"]
    y_distance = roi_results_distance[roi]["y_fixed"]

    # Residual: distance - sqrt(area)
    residual = y_distance - np.sqrt(y_areal)
    residuals_by_roi[roi] = residual

    # Standard error from CI for both y_areal and y_distance
    y_lower_areal = roi_results_area[roi]["y_lower"]
    y_upper_areal = roi_results_area[roi]["y_upper"]
    se_areal = (y_upper_areal - y_lower_areal) / (2 * z)

    y_lower_distance = roi_results_distance[roi]["y_lower"]
    y_upper_distance = roi_results_distance[roi]["y_upper"]
    se_distance = (y_upper_distance - y_lower_distance) / (2 * z)

    # Propagate SE through sqrt(y_areal)
    # For f(x) = sqrt(x), Var(f) ≈ (1/(2*sqrt(x)))^2 * Var(x)
    se_sqrt_areal = se_areal / (2 * np.sqrt(y_areal))

    # Residual SE: combine variances assuming independence
    se_residual = np.sqrt(se_distance**2 + se_sqrt_areal**2)
    se_by_roi[roi] = se_residual

# Assign rainbow colors
n_rois = len(residuals_by_roi)
colors = cm.rainbow(np.linspace(0, 1, n_rois))

# Plot residuals with SE shading
plt.figure(figsize=(10,9))
for i, (roi, resid) in enumerate(residuals_by_roi.items()):
    se = se_by_roi.get(roi, None)
    if se is not None:
        plt.fill_between(x_common, resid - se, resid + se,
                         color=colors[i], alpha=0.2, lw=0)  # shaded band
    plt.plot(x_common, resid, label=roi, lw=2, color=colors[i])  # residual line

plt.axhline(0, color='k', linestyle='--', lw=2)
plt.xlim(0.1, 8)
plt.ylim(-3.5, 17.5)
plt.xlabel("Eccentricity (deg)", fontsize=30)
plt.ylabel("Residuals (mm/deg)", fontsize=30)

# Log scale with exact ticks
ax = plt.gca()
ax.set_xscale('log')
ax.set_xticks([0.1, 1, 8])
ax.set_xticklabels(['0.1', '1', '8'], fontsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#plt.legend(fontsize=18, ncol=2, loc='right')
plt.tight_layout()
plt.savefig("sqrt_residuals_with_se.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Plot V1 distance squared and area

roi = "V1"
z = 1.96  # for ~95% CI

# Extract data for V1
x = roi_results_area[roi]["x_common"]

# Area
y_area = roi_results_area[roi]["y_fixed"]
y_area_lower = roi_results_area[roi]["y_lower"]
y_area_upper = roi_results_area[roi]["y_upper"]
se_area = (y_area_upper - y_area_lower) / (2 * z)

# Distance (squared)
y_dist = roi_results_distance[roi]["y_fixed"] ** 2
y_dist_lower = roi_results_distance[roi]["y_lower"] ** 2
y_dist_upper = roi_results_distance[roi]["y_upper"] ** 2
se_dist = (y_dist_upper - y_dist_lower) / (2 * z)

# Plot
plt.figure(figsize=(8,8))

# Area curve with shaded SE
line_area, = plt.plot(x, y_area, color='blue', lw=2, label='Area')
plt.fill_between(x, y_area - se_area, y_area + se_area, color='blue', alpha=0.2)

# Distance^2 curve with shaded SE
line_dist, = plt.plot(x, y_dist, color='red', lw=2, label='Distance²')
plt.fill_between(x, y_dist - se_dist, y_dist + se_dist, color='red', alpha=0.2)

plt.xlabel("Eccentricity (deg)", fontsize=30)
plt.ylabel("CM (mm²/deg²)", fontsize=30)
#plt.title("V1: CM vs Eccentricity", fontsize=18)
plt.legend(handles=[line_dist, line_area],fontsize=22)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

#plt.xscale('log')
plt.xlim(0, 8)
#plt.xticks([0.1, 1, 8], ['0.1', '1', '8'])
plt.savefig("V1_dist_sq.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Plot V1 distance and sqrt(area)

roi = "V1"
z = 1.96  # for ~95% CI

# Extract data for V1
x = roi_results_area[roi]["x_common"]

# Distance (keep original units)
y_dist = roi_results_distance[roi]["y_fixed"]
y_dist_lower = roi_results_distance[roi]["y_lower"]
y_dist_upper = roi_results_distance[roi]["y_upper"]
se_dist = (y_dist_upper - y_dist_lower) / (2 * z)

# Area (square root)
y_area_sqrt = np.sqrt(roi_results_area[roi]["y_fixed"])
y_area_lower_sqrt = np.sqrt(roi_results_area[roi]["y_lower"])
y_area_upper_sqrt = np.sqrt(roi_results_area[roi]["y_upper"])
se_area_sqrt = (y_area_upper_sqrt - y_area_lower_sqrt) / (2 * z)

# Plot
plt.figure(figsize=(8,8))

# Distance curve with shaded SE
plt.plot(x, y_dist, color='red', lw=2, label='Distance')
plt.fill_between(x, y_dist - se_dist, y_dist + se_dist, color='red', alpha=0.2)

# sqrt(Area) curve with shaded SE
plt.plot(x, y_area_sqrt, color='blue', lw=2, label='√Area')
plt.fill_between(x, y_area_sqrt - se_area_sqrt, y_area_sqrt + se_area_sqrt, color='blue', alpha=0.2)

plt.xlabel("Eccentricity (deg)", fontsize=30)
plt.ylabel("CM (mm/deg)", fontsize=30)
#plt.title("V1: CM vs Eccentricity (Distance vs √Area)", fontsize=18)
plt.legend(fontsize=22)
plt.xlim(0, 8)

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)

#plt.xscale('log')
#plt.xticks([0.1, 1, 8], ['0.1', '1', '8'])
plt.savefig("V1_area_sqrt.png", dpi=300, bbox_inches='tight')
plt.show()
